In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import timeit
import random
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy
# Import the requests library.
import requests
# Import the API key.
from config import weather_api_key
# Import the datetime module from the datetime library.
from datetime import datetime
import time

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

747

In [5]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [9]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # City Description
        city_description = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | kuandian
Processing Record 2 of Set 1 | tefe
Processing Record 3 of Set 1 | sao felix do xingu
Processing Record 4 of Set 1 | bluff
Processing Record 5 of Set 1 | ostrovnoy
Processing Record 6 of Set 1 | ushuaia
Processing Record 7 of Set 1 | carnarvon
Processing Record 8 of Set 1 | talnakh
Processing Record 9 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 10 of Set 1 | saint-joseph
Processing Record 11 of Set 1 | longyearbyen
Processing Record 12 of Set 1 | deputatskiy
Processing Record 13 of Set 1 | ahipara
Processing Record 14 of Set 1 | chama
Processing Record 15 of Set 1 | fevralsk
City not found. Skipping...
Processing Record 16 of Set 1 | gushikawa
Processing Record 17 of Set 1 | mataura
Processing Record 18 of Set 1 | rolim de moura
City not found. Skipping...
Processing Record 19 of Set 1 | rudbar
Processing Record 20 of Set 1 | port elizabeth
Processing Record 21 of 

In [11]:
# Convert the array of dictionaries to a Pandas DataFrame.
WeatherPy_df = pd.DataFrame(city_data)
WeatherPy_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Kuandian,CN,40.7286,124.7847,83.28,68,100,3.85,overcast clouds
1,Tefe,BR,-3.3542,-64.7114,75.51,88,64,2.44,broken clouds
2,Sao Felix Do Xingu,BR,-6.6447,-51.9950,72.18,52,76,1.92,broken clouds
3,Bluff,NZ,-46.6000,168.3333,56.80,64,34,6.31,scattered clouds
4,Ostrovnoy,RU,68.0531,39.5131,43.29,95,100,25.32,overcast clouds
5,Ushuaia,AR,-54.8000,-68.3000,37.06,75,90,10.36,overcast clouds
6,Carnarvon,AU,-24.8667,113.6333,62.67,56,91,11.50,overcast clouds
7,Talnakh,RU,69.4865,88.3972,67.82,74,100,22.06,light rain
8,Saint-Joseph,RE,-21.3667,55.6167,64.78,68,19,6.98,few clouds
9,Longyearbyen,SJ,78.2186,15.6401,44.44,87,75,8.05,broken clouds


In [12]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
WeatherPy_df.to_csv(output_data_file, index_label="City_ID")